In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold,GridSearchCV,RandomizedSearchCV,cross_val_score
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier,RandomForestRegressor,BaggingRegressor,AdaBoostRegressor,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression,LogisticRegression,Lasso, Ridge
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
import sklearn.metrics as metrics
from sklearn.metrics import r2_score,roc_auc_score,classification_report,mean_squared_error,accuracy_score,confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('D:/datasets+minipro/analytics vidhya datasets/janata hack p3 hr analytics/train.csv')
dfte=pd.read_csv('D:/datasets+minipro/analytics vidhya datasets/janata hack p3 hr analytics/test.csv')

In [3]:
df.isnull().sum()

enrollee_id                  0
city                         0
city_development_index       0
gender                    4098
relevent_experience          0
enrolled_university        342
education_level            457
major_discipline          2838
experience                  59
company_size              4779
company_type              5039
last_new_job               367
training_hours               0
target                       0
dtype: int64

In [4]:
df1=df.copy() # backup

#### Work on copy df1

In [5]:
df1.drop(['enrollee_id','city'],1,inplace=True)

In [6]:
df['gender'].value_counts()

Male      12884
Female     1188
Other       189
Name: gender, dtype: int64

In [7]:
# Gender feature imputed with mode & converted to categorical
df1['gender']=df1['gender'].map({'Male':1,'Female':0,'Other':2})
df1['gender']=df1['gender'].fillna(1).astype('int')

# enrolled_university feature imputed with mode & converted to categorical 
df1['enrolled_university']=df1['enrolled_university'].fillna('no_enrollment')
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df1['enrolled_university']=le.fit_transform(df1['enrolled_university'])

df1['education_level']=df1['education_level'].fillna('Graduate')
df1['education_level']=le.fit_transform(df1['education_level'])

df1['major_discipline']=df1['major_discipline'].fillna('STEM')
df1['major_discipline']=le.fit_transform(df1['major_discipline'])

df1['experience'].replace({'<1':'1','>20':'20'},inplace=True)
df1['experience']=df1['experience'].fillna('20')
df1['experience']=df1['experience'].astype('int')

df1['company_size'].replace({'<10':'10','10/49':'30','50-99':'75','100-500':'300','500-999':'750',
                            '1000-4999':'3000','5000-9999':'7500','10000+':'10000'},inplace=True)

df1['company_size']=df1['company_size'].fillna('75')
df1['company_size']=df1['company_size'].astype('int')

df1['last_new_job'].replace({'>4':4,'never':0},inplace=True)

df1['last_new_job']=df1['last_new_job'].fillna(1)

df1['last_new_job']=df1['last_new_job'].astype('int')

df1['company_type']=df1['company_type'].fillna('Pvt Ltd')
df1['company_type']=le.fit_transform(df1['company_type'])

df1['relevent_experience']=le.fit_transform(df1['relevent_experience'])

In [8]:
df1['target'].value_counts(normalize=True)*100# Imbalanced dataset

0    86.79122
1    13.20878
Name: target, dtype: float64

In [9]:
# Lets try without any over sampling,undersampling or SMOTE

In [10]:
#set seed for same results everytime
seed=0
import sklearn.ensemble as ensemble
import sklearn.metrics as metrics

X=df1.drop('target',1)
y=df1['target']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state =1)

#declare the models
lr = LogisticRegression()
rf=RandomForestClassifier()
adb=ensemble.AdaBoostClassifier()
bgc=ensemble.BaggingClassifier()
gnb = GaussianNB()
knn=KNeighborsClassifier()
dt = DecisionTreeClassifier()
# ab_rf = AdaBoostClassifier(base_estimator=rf,random_state=0)
# ab_dt = AdaBoostClassifier(base_estimator=dt,random_state=0)
# ab_nb=  AdaBoostClassifier(base_estimator=gnb,random_state=0)
# ab_lr=  AdaBoostClassifier(base_estimator=lr,random_state=0)
bgcl_lr = BaggingClassifier(base_estimator=lr, random_state=0)

# ,ab_rf,ab_dt,ab_nb,ab_lr,bgcl_lr

models=[lr,rf,adb,bgc,gnb,knn,dt,bgcl_lr]
sctr,scte,auc,ps,rs=[],[],[],[],[]
def ens(X_train,X_test, y_train, y_test):
    for model in models:
            model.fit(X_train, y_train)
            y_test_pred = model.predict(X_test)
            y_test_pred_new=model.predict_proba(X_test)
            y_test_pred_new=y_test_pred_new[:,1]
            train_score=model.score(X_train,y_train)
            test_score=model.score(X_test,y_test)
            p_score=metrics.precision_score(y_test,y_test_pred)
            r_score=metrics.recall_score(y_test,y_test_pred)
            
            ac=metrics.roc_auc_score(y_test,y_test_pred_new)
            
            sctr.append(train_score)
            scte.append(test_score)
            ps.append(p_score)
            rs.append(r_score)
            auc.append(ac)
    return sctr,scte,auc,ps,rs
ens(X_train,X_test, y_train, y_test)
# 'ab_rf','ab_dt','ab_nb','ab_lr','bgcl_lr'
ensemble=pd.DataFrame({'names':['Logistic Regression','Random Forest','Ada boost','Bagging',
                                'Naive-Bayes','KNN','Decistion Tree',
                                'bagged LR'],
                       'auc_score':auc,'training':sctr,'testing':scte,'precision':ps,'recall':rs})
ensemble=ensemble.sort_values(by='auc_score',ascending=False).reset_index(drop=True)
ensemble

,names,auc_score,training,testing,precision,recall
0,Ada boost,0.639299,0.867298,0.870643,0.000000,0.000000
1,Random Forest,0.591168,0.998230,0.862745,0.206897,0.025532
2,bagged LR,0.589443,0.866889,0.872004,0.000000,0.000000
3,Naive-Bayes,0.588875,0.850344,0.857843,0.267857,0.063830
4,Logistic Regression,0.584952,0.866889,0.872004,0.000000,0.000000
5,Bagging,0.544813,0.978144,0.854575,0.214286,0.051064
6,KNN,0.528065,0.873834,0.857843,0.166667,0.027660
7,Decistion Tree,0.518036,0.998298,0.766612,0.152603,0.180851


### Manipulating test data:

In [11]:
df2=dfte.copy() ## backup of dfte

In [12]:

df2.drop(['city','enrollee_id'],1,inplace=True)

# Gender feature imputed with mode & converted to categorical
df2['gender']=df2['gender'].map({'Male':1,'Female':0,'Other':2})
df2['gender']=df2['gender'].fillna(1).astype('int')

# enrolled_university feature imputed with mode & converted to categorical 
df2['enrolled_university']=df2['enrolled_university'].fillna('no_enrollment')
df2['enrolled_university']=le.fit_transform(df2['enrolled_university'])

df2['education_level']=df2['education_level'].fillna('Graduate')
df2['education_level']=le.fit_transform(df2['education_level'])

df2['major_discipline']=df2['major_discipline'].fillna('STEM')
df2['major_discipline']=le.fit_transform(df2['major_discipline'])

df2['experience'].replace({'<1':'1','>20':'20'},inplace=True)
df2['experience']=df2['experience'].fillna('20')
df2['experience']=df2['experience'].astype('int')

df2['company_size'].replace({'<10':'10','10/49':'30','50-99':'75','100-500':'300','500-999':'750',
                            '1000-4999':'3000','5000-9999':'7500','10000+':'10000'},inplace=True)

df2['company_size']=df2['company_size'].fillna('75')
df2['company_size']=df2['company_size'].astype('int')

df2['last_new_job'].replace({'>4':4,'never':0},inplace=True)

df2['last_new_job']=df2['last_new_job'].fillna(1)

df2['last_new_job']=df2['last_new_job'].astype('int')

df2['company_type']=df2['company_type'].fillna('Pvt Ltd')
df2['company_type']=le.fit_transform(df2['company_type'])

df2['relevent_experience']=le.fit_transform(df2['relevent_experience'])




In [13]:

## Making predictions:
from sklearn.ensemble import AdaBoostClassifier
adb=AdaBoostClassifier()

adb.fit(X,y)
y_test_pred_adb=adb.predict_proba(df2)[:,1]
finalpred=pd.concat([dfte['enrollee_id'],pd.DataFrame(y_test_pred_adb,columns=['target'])],1)
finalpred.to_csv("D:/datasets+minipro/analytics vidhya datasets/janata hack p3 hr analytics/adbpred.csv",index=False)

# SMOTE:

In [14]:
from imblearn.over_sampling import SMOTE

oversample = SMOTE()
X_smote, y_smote = oversample.fit_resample(X, y)

In [15]:
#set seed for same results everytime
seed=0
import sklearn.ensemble as ensemble
import sklearn.metrics as metrics

X=X_smote
y=y_smote
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state =1)

#declare the models
lr = LogisticRegression()
rf=RandomForestClassifier()
adb=ensemble.AdaBoostClassifier()
bgc=ensemble.BaggingClassifier()
gnb = GaussianNB()
knn=KNeighborsClassifier()
dt = DecisionTreeClassifier()
# ab_rf = AdaBoostClassifier(base_estimator=rf,random_state=0)
# ab_dt = AdaBoostClassifier(base_estimator=dt,random_state=0)
# ab_nb=  AdaBoostClassifier(base_estimator=gnb,random_state=0)
# ab_lr=  AdaBoostClassifier(base_estimator=lr,random_state=0)
bgcl_lr = BaggingClassifier(base_estimator=lr, random_state=0)

# ,ab_rf,ab_dt,ab_nb,ab_lr,bgcl_lr

models=[lr,rf,adb,bgc,gnb,knn,dt,bgcl_lr]
sctr,scte,auc,ps,rs=[],[],[],[],[]
def ens(X_train,X_test, y_train, y_test):
    for model in models:
            model.fit(X_train, y_train)
            y_test_pred = model.predict(X_test)
            y_test_pred_new=model.predict_proba(X_test)
            y_test_pred_new=y_test_pred_new[:,1]
            train_score=model.score(X_train,y_train)
            test_score=model.score(X_test,y_test)
            p_score=metrics.precision_score(y_test,y_test_pred)
            r_score=metrics.recall_score(y_test,y_test_pred)
            
            ac=metrics.roc_auc_score(y_test,y_test_pred_new)
            
            sctr.append(train_score)
            scte.append(test_score)
            ps.append(p_score)
            rs.append(r_score)
            auc.append(ac)
    return sctr,scte,auc,ps,rs
ens(X_train,X_test, y_train, y_test)
# 'ab_rf','ab_dt','ab_nb','ab_lr','bgcl_lr'
ensemble=pd.DataFrame({'names':['Logistic Regression','Random Forest','Ada boost','Bagging',
                                'Naive-Bayes','KNN','Decistion Tree',
                                'bagged LR'],
                       'auc_score':auc,'training':sctr,'testing':scte,'precision':ps,'recall':rs})
ensemble=ensemble.sort_values(by='auc_score',ascending=False).reset_index(drop=True)
ensemble

,names,auc_score,training,testing,precision,recall
0,Random Forest,0.921682,0.998588,0.848761,0.840810,0.856282
1,Bagging,0.919451,0.988429,0.860056,0.876667,0.834391
2,KNN,0.866786,0.858712,0.771729,0.707305,0.918464
3,Ada boost,0.811116,0.738331,0.730311,0.720123,0.743655
4,Decistion Tree,0.807264,0.998588,0.806715,0.792505,0.825190
5,Naive-Bayes,0.672831,0.623794,0.617352,0.607736,0.638008
6,bagged LR,0.661305,0.611438,0.608566,0.601483,0.617703
7,Logistic Regression,0.659812,0.607947,0.602918,0.595100,0.616434


# RF:

In [16]:
from sklearn.ensemble import RandomForestClassifier

rf=RandomForestClassifier()

rf.fit(X_smote,y_smote)

y_test_pred_rf=rf.predict_proba(df2)[:,1]
finalpred=pd.concat([dfte['enrollee_id'],pd.DataFrame(y_test_pred_rf,columns=['target'])],1)
finalpred.to_csv("D:/datasets+minipro/analytics vidhya datasets/janata hack p3 hr analytics/rfpred.csv",index=False)

# DT:

In [17]:
X=X_smote
y=y_smote
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state =0)
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

dt = DecisionTreeClassifier()
dt_params = {'max_depth':np.arange(1,10), 'min_samples_leaf':np.arange(2,30), 'criterion':['entropy','gini']}
rscv = RandomizedSearchCV(dt, dt_params, cv=5, scoring='roc_auc')
rscv.fit(X, y)
print(rscv.best_params_)
rscv_best_DT=rscv.best_params_

DT=DecisionTreeClassifier(**rscv_best_DT)
DT.fit(X_smote,y_smote)
y_test_pred_DT=DT.predict_proba(df2)[:,1]
finalpred=pd.concat([dfte['enrollee_id'],pd.DataFrame(y_test_pred_DT,columns=['target'])],1)
finalpred.to_csv("D:/datasets+minipro/analytics vidhya datasets/janata hack p3 hr analytics/DTpred.csv",index=False)

{'min_samples_leaf': 6, 'max_depth': 9, 'criterion': 'gini'}


# XGB classifier:

In [18]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [19]:
xgb=XGBClassifier(learning_rate=0.09,n_estimators=125,max_depth=4,min_child_weight=4,colsample_bytree=0.5,reg_alpha=0.000001 )

In [20]:
xgb.fit(X_smote,y_smote)
y_test_pred_xgb=xgb.predict_proba(df2)[:,1]
finalpred=pd.concat([dfte['enrollee_id'],pd.DataFrame(y_test_pred_xgb,columns=['target'])],1)
finalpred.to_csv("D:/datasets+minipro/analytics vidhya datasets/janata hack p3 hr analytics/xgbpred.csv",index=False)

# using get_dummies:

In [21]:
df3=df.copy() # backup

In [22]:
from sklearn.preprocessing import OneHotEncoder

In [23]:
ohe=OneHotEncoder(sparse=False)

In [24]:
df3['enrolled_university'].fillna('no_enrollment',inplace=True)
df3['last_new_job'].fillna('1',inplace=True)
df3['education_level'].fillna('Graduate',inplace=True)
df3['experience'].fillna('>20',inplace=True)

more=['gender','company_size','company_type','major_discipline'] 
#more is for features having missing values greater than 15%
for i in range(len(more)):
    df3[more[i]].fillna('missing',axis=0,inplace=True)

df3.drop(['enrollee_id','city'],1,inplace=True)

df3['experience'].replace({'<1':'1','>20':'21'},inplace=True)
df3['company_size'].replace({'<10':'10','10/49':'30','50-99':'75','100-500':'300','500-999':'750',
                            '1000-4999':'3000','5000-9999':'7500','10000+':'10000'},inplace=True)
df3['last_new_job'].replace({'>4':5,'never':0},inplace=True)

df3=pd.get_dummies(data=df3,drop_first=True)

In [25]:
df4=dfte.copy() # backup of test data is used

In [26]:
df4['enrolled_university'].fillna('no_enrollment',inplace=True)
df4['last_new_job'].fillna('1',inplace=True)
df4['education_level'].fillna('Graduate',inplace=True)
df4['experience'].fillna('>20',inplace=True)

more=['gender','company_size','company_type','major_discipline'] 
#more is for features having missing values greater than 15%
for i in range(len(more)):
    df4[more[i]].fillna('missing',axis=0,inplace=True)
    
df4.drop(['enrollee_id','city'],1,inplace=True)
df4['experience'].replace({'<1':'1','>20':'21'},inplace=True)
df4['company_size'].replace({'<10':'10','10/49':'30','50-99':'75','100-500':'300','500-999':'750',
                            '1000-4999':'3000','5000-9999':'7500','10000+':'10000'},inplace=True)
df4['last_new_job'].replace({'>4':5,'never':0},inplace=True)

In [27]:
df4=pd.get_dummies(data=df4,drop_first=True)

In [28]:
df4.shape

(15021, 57)

In [29]:
X=df3.drop('target',1)
y=df3['target']
X_smote1, y_smote1 = oversample.fit_resample(X, y)

In [30]:
#set seed for same results everytime
# seed=0
# import sklearn.ensemble as ensemble
# import sklearn.metrics as metrics

# X=X_smote1
# y=y_smote1
# X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state =1)

# #declare the models
# lr = LogisticRegression()
# rf=RandomForestClassifier()
# adb=ensemble.AdaBoostClassifier()
# bgc=ensemble.BaggingClassifier()
# gnb = GaussianNB()
# knn=KNeighborsClassifier()
# dt = DecisionTreeClassifier()
# # ab_rf = AdaBoostClassifier(base_estimator=rf,random_state=0)
# # ab_dt = AdaBoostClassifier(base_estimator=dt,random_state=0)
# # ab_nb=  AdaBoostClassifier(base_estimator=gnb,random_state=0)
# # ab_lr=  AdaBoostClassifier(base_estimator=lr,random_state=0)
# bgcl_lr = BaggingClassifier(base_estimator=lr, random_state=0)

# # ,ab_rf,ab_dt,ab_nb,ab_lr,bgcl_lr

# models=[lr,rf,adb,bgc,gnb,knn,dt,bgcl_lr]
# sctr,scte,auc,ps,rs=[],[],[],[],[]
# def ens(X_train,X_test, y_train, y_test):
#     for model in models:
#             model.fit(X_train, y_train)
#             y_test_pred = model.predict(X_test)
#             y_test_pred_new=model.predict_proba(X_test)
#             y_test_pred_new=y_test_pred_new[:,1]
#             train_score=model.score(X_train,y_train)
#             test_score=model.score(X_test,y_test)
#             p_score=metrics.precision_score(y_test,y_test_pred)
#             r_score=metrics.recall_score(y_test,y_test_pred)
            
#             ac=metrics.roc_auc_score(y_test,y_test_pred_new)
            
#             sctr.append(train_score)
#             scte.append(test_score)
#             ps.append(p_score)
#             rs.append(r_score)
#             auc.append(ac)
#     return sctr,scte,auc,ps,rs
# ens(X_train,X_test, y_train, y_test)
# # 'ab_rf','ab_dt','ab_nb','ab_lr','bgcl_lr'
# ensemble=pd.DataFrame({'names':['Logistic Regression','Random Forest','Ada boost','Bagging',
#                                 'Naive-Bayes','KNN','Decistion Tree',
#                                 'bagged LR'],
#                        'auc_score':auc,'training':sctr,'testing':scte,'precision':ps,'recall':rs})
# ensemble=ensemble.sort_values(by='auc_score',ascending=False).reset_index(drop=True)
# ensemble

In [31]:
xgb.fit(X_smote1,y_smote1)
y_test_pred_xgb=xgb.predict_proba(df4)[:,1]
finalpred=pd.concat([dfte['enrollee_id'],pd.DataFrame(y_test_pred_xgb,columns=['target'])],1)
finalpred.to_csv("D:/datasets+minipro/analytics vidhya datasets/janata hack p3 hr analytics/xgbpred.csv",index=False)

# combination of LE & ohe

In [32]:
df5=df.copy() # backup

df5['enrolled_university'].fillna('no_enrollment',inplace=True)
df5['last_new_job'].fillna('1',inplace=True)
df5['education_level'].fillna('Graduate',inplace=True)
df5['experience'].fillna('>20',inplace=True)

more=['gender','company_size','company_type','major_discipline'] 
#more is for features having missing values greater than 15%
for i in range(len(more)):
    df5[more[i]].fillna('missing',axis=0,inplace=True)

df5.drop(['enrollee_id','city'],1,inplace=True)

df5['experience'].replace({'<1':'0','>20':'21'},inplace=True)
df5['company_size'].replace({'<10':'10','10/49':'30','50-99':'75','100-500':'300','500-999':'750',
                            '1000-4999':'3000','5000-9999':'7500','10000+':'10000'},inplace=True)
df5['last_new_job'].replace({'>4':5,'never':0},inplace=True)

In [33]:
df5['enrolled_university'].replace({'no_enrollment':0,'Part time course':1,'Full time course':2},inplace=True)

df5['relevent_experience'].replace({'No relevent experience':0,'Has relevent experience':1},inplace=True)

df5['company_size'].replace({'missing':75},inplace=True)

df5['company_size']=df5['company_size'].astype('int')

df5['company_size']=df5.groupby('company_type').company_size.apply(lambda x:x.fillna(x.median()))

In [34]:
df5=pd.get_dummies(data=df5,drop_first=True)

In [35]:

df6=dfte.copy() # backup of test data is used

df6['enrolled_university'].fillna('no_enrollment',inplace=True)
df6['last_new_job'].fillna('1',inplace=True)
df6['education_level'].fillna('Graduate',inplace=True)
df6['experience'].fillna('>20',inplace=True)

more=['gender','company_size','company_type','major_discipline'] 
#more is for features having missing values greater than 15%
for i in range(len(more)):
    df6[more[i]].fillna('missing',axis=0,inplace=True)
    
df6.drop(['enrollee_id','city'],1,inplace=True)
df6['experience'].replace({'<1':'0','>20':'21'},inplace=True)
df6['company_size'].replace({'<10':'10','10/49':'30','50-99':'75','100-500':'300','500-999':'750',
                            '1000-4999':'3000','5000-9999':'7500','10000+':'10000'},inplace=True)
df6['last_new_job'].replace({'>4':5,'never':0},inplace=True)



In [36]:
df6['enrolled_university'].replace({'no_enrollment':0,'Part time course':1,'Full time course':2},inplace=True)

df6['relevent_experience'].replace({'No relevent experience':0,'Has relevent experience':1},inplace=True)

df6['company_size'].replace({'missing':75},inplace=True)

df6['company_size']=df6['company_size'].astype('int')

df6['company_size']=df6.groupby('company_type').company_size.apply(lambda x:x.fillna(x.median()))

In [37]:
df6=pd.get_dummies(data=df6,drop_first=True)

In [38]:
X=df5.drop('target',1)
y=df5['target']
# X_smote2, y_smote2 = oversample.fit_resample(X, y)

### This above setting which for df5 and df6 is a combination of label encoding and get_dummies is giving best accuracy.

# Best accuracy with xgb classifier:

In [39]:
# XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#               colsample_bynode=1, colsample_bytree=0.5, gamma=0,
#               learning_rate=0.09, max_delta_step=0, max_depth=4,
#               min_child_weight=4, missing=None, n_estimators=125, n_jobs=1,
#               nthread=None, objective='binary:logistic', random_state=0,
#               reg_alpha=1e-06, reg_lambda=1, scale_pos_weight=1, seed=None,
#               silent=None, subsample=1, verbosity=1)

In [40]:
xgb.fit(X,y)
y_test_pred_xgb=xgb.predict_proba(df6)[:,1]
finalpred=pd.concat([dfte['enrollee_id'],pd.DataFrame(y_test_pred_xgb,columns=['target'])],1)
finalpred.to_csv("D:/datasets+minipro/analytics vidhya datasets/janata hack p3 hr analytics/xgbpred2.csv",index=False)

# TP:

In [56]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X,y)

AttributeError: 'GridSearchCV' object has no attribute 'grid_scores_'

In [59]:
 gsearch1.best_params_, gsearch1.best_score_
#({'max_depth': 3, 'min_child_weight': 12}, 0.6536800078557696)

({'max_depth': 3, 'min_child_weight': 1}, 0.6536800078557696)

In [64]:
# param_test2b = {
#  'min_child_weight':[6,8,10,12]
# }
# gsearch2b = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=125, max_depth=3,
#  min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
#  objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
#  param_grid = param_test2b, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
# gsearch2b.fit(X,y)

# gsearch2b.best_params_, gsearch2b.best_score_

# param_test3 = {
#  'gamma':[i/10.0 for i in range(0,5)]
# }
# gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=125, max_depth=3,
#  min_child_weight=12, gamma=0, subsample=0.8, colsample_bytree=0.8,
#  objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
#  param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
# gsearch3.fit(X,y)
# gsearch3.best_params_, gsearch3.best_score_

# param_test7 = {
#  'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
# }
# gsearch7 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=125, max_depth=3,
#  min_child_weight=12, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
#  objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
#  param_grid = param_test7, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
# gsearch7.fit(X,y)
# gsearch7.best_params_, gsearch7.best_score_

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=0.8, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=2,
                                     missing=None, n_estimators=125, n_jobs=1,
                                     nthread=4, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=27, silent=None,
                                     subsample=0.8, verbosity=1),
             iid=False, n_jobs=4,
             param_grid={'min_child_weight': [6, 8, 10, 12]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='roc_auc', verbose=0)

In [78]:
# xgb= XGBClassifier( learning_rate =0.1,reg_alpha=0.006, n_estimators=125, max_depth=3,
#  min_child_weight=12, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
#  objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27)

In [79]:
xgb.fit(X,y)
y_test_pred_xgb=xgb.predict_proba(df6)[:,1]
finalpred=pd.concat([dfte['enrollee_id'],pd.DataFrame(y_test_pred_xgb,columns=['target'])],1)
finalpred.to_csv("D:/datasets+minipro/analytics vidhya datasets/janata hack p3 hr analytics/xgbpred2.csv",index=False)